In [20]:
import re
import pandas as pd
import numpy as np
# from mlutils import dataset, connector
import scipy.stats
from scipy.stats import *
import scipy.stats as stats

from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import warnings
import statsmodels.api as sm 
import seaborn as sns
import pylab as py 
warnings.filterwarnings('ignore')
%matplotlib inline

In [21]:
def standarise(y,pct,pct_lower):
    sc = StandardScaler() 
    y.sort()
    len_y = len(y)
    y = y[int(pct_lower * len_y):int(len_y * pct)]
    len_y = len(y)
    yy=([[x] for x in y])
    sc.fit(yy)
    y_std =sc.transform(yy)
    y_std = y_std.flatten()
    return y_std,len_y,y

In [35]:
def fit_distribution(column,pct,pct_lower):
    # Set up list of candidate distributions to use
    # See https://docs.scipy.org/doc/scipy/reference/stats.html for more
    y_std,size,y_org = standarise(column,pct,pct_lower)
    dist_names = ['weibull_min','norm','weibull_max','beta',
                 'invgauss','uniform','gamma','expon', 'lognorm','pearson3','triang','erlang']

    chi_square_statistics = []
    # 11 bins
    percentile_bins = np.linspace(0,100,11)
    percentile_cutoffs = np.percentile(y_std, percentile_bins)
    observed_frequency, bins = (np.histogram(y_std, bins=percentile_cutoffs))
    cum_observed_frequency = np.cumsum(observed_frequency)

    # Loop through candidate distributions

    for distribution in dist_names:
        # Set up distribution and get fitted distribution parameters
        dist = getattr(scipy.stats, distribution)
        param = dist.fit(y_std)
        print("{}\n{}\n".format(dist, param))
      

        # Get expected counts in percentile bins
        # cdf of fitted sistrinution across bins
        cdf_fitted = dist.cdf(percentile_cutoffs, *param)
        expected_frequency = []
        for bin in range(len(percentile_bins)-1):
            expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
            expected_frequency.append(expected_cdf_area)

        # Chi-square Statistics
        expected_frequency = np.array(expected_frequency) * size
        cum_expected_frequency = np.cumsum(expected_frequency)
        ss = round(sum (((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency),6)
        chi_square_statistics.append(ss)


    #Sort by minimum ch-square statistics
    results = pd.DataFrame()
    results['Distribution'] = dist_names
    results['chi_square'] = chi_square_statistics
    results.sort_values(['chi_square'], inplace=True)


    print('\nDistributions listed by Betterment of fit:')
    print('............................................')
    print(results)

## Service times

In [38]:
import read_data as rd

In [39]:
filenames = ["KevinYeData.txt",
             "PatrickQData.txt",
             "TamaHoareData.txt",
             "dataviviandong.txt"]
results = rd.ReadData(filenames=filenames)
AllArrivals = results[0]
AllServices = results[1]

In [40]:
fit_distribution(AllArrivals,0.99,0.01)

(0.7015384049768638, -0.9894365309023049, 1.2006676444778561)

(0.0, 1.0)

(0.7019179793807708, 3.26754168282175, 1.2352709611338757)

(0.5869141736933967, 2.0230692974358018, -0.9894365309023049, 5.611707278987666)

(1.3774125862615234, -1.133748597072893, 0.8230934376760495)

(-0.9894365309023048, 4.256978213724054)

(0.8667586476420364, -0.989436530902305, 0.8160179858473148)

(-0.9894365309023048, 0.9894365309023048)

(1.2285098631304354, -1.040346141653061, 0.5785087400178923)

(2.416641101443037, -0.007126039833642914, 1.1869459535476108)

(1.2422294892673428e-07, -0.9894371622132272, 4.384475250138744)

(0.34939810853283576, -0.9894365309023049, 1.0870659237131632)


Distributions listed by Betterment of fit:
............................................
   Distribution   chi_square
9      pearson3     9.155252
0   weibull_min    13.245287
3          beta    21.708357
8       lognorm    25.596288
4      invgauss    29.389634
7         expon    29.515278
6         gamma    48.3593

In [41]:
fit_distribution(AllServices,0.99,0.01)

(1.352589278269006, -1.3447015820584827, 1.467232807050587)

(2.241459734258991e-16, 1.0)

(0.721662688150412, 3.1455922485465155, 1.2433305622072686)

(1.394142397577749, 7.296709096440065, -1.3449459239135244, 8.392743503955604)

(0.4273306502409351, -1.68320695484702, 3.9388870495280726)

(-1.340184443719104, 4.485776692265619)

(1.6604935980397486, -1.3524386238116077, 0.8144683757801132)

(-1.340184443719104, 1.3401844437191042)

(0.6415966921638407, -1.6127636448548777, 1.3280431345013661)

(1.5520636624046062, 7.17308070388293e-17, 1.0495415246778275)

(0.03711977537648216, -1.3648084498774897, 4.646597079496283)

(1.6605072541820394, -1.352434083723125, 0.814480138474973)


Distributions listed by Betterment of fit:
............................................
   Distribution   chi_square
3          beta     1.582931
0   weibull_min     2.017339
9      pearson3     3.274222
6         gamma     3.274604
11       erlang     3.274684
4      invgauss     8.160098
8       lognorm   

## Fitting different Distributions and checking Goodness of fit based on Chi-square Statistics.

Based on the Chi-sqaure Statistics, it suggests that Pearson3 is the best in approximating ‘inter-arrival time’ Data.

We can also see that beta distribution is the best fit for service times.